In [1]:
import os 
import numpy as np
import pandas as pd
import pickle 
from astropy.table import Table

In [2]:
train_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_HEAD.FITS'
train_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/train_final_master_PHOT.FITS'
test_data_head_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_HEAD.FITS'
test_data_phot_path = '/sps/lsst/users/bbiswas/data/kilonova_datasets/test_final_master_PHOT.FITS'

In [3]:
df_header = Table.read(train_data_head_path, format='fits').to_pandas()
df_phot = Table.read(train_data_phot_path, format='fits').to_pandas()

In [4]:
from kndetect.utils import load_pcs, get_event_type, get_data_dir_path
from kndetect.features import extract_features_all_lightcurves
from kndetect.training import append_y_true_col

In [5]:
pcs = load_pcs()
data_dir = get_data_dir_path()

In [6]:
use_already_trained_features = False
use_already_trained_models = False
mimic_alerts = True
save_data = True

if mimic_alerts:
    sub_directory = "partial"
else: 
    sub_directory = "complete"

In [7]:
if use_already_trained_features:
    train_features_df = pd.read_csv(os.path.join(data_dir, sub_directory, "train_features.csv"),index_col=0)

else:
    train_features_df = extract_features_all_lightcurves(df_phot, 
                                                         "SNID", 
                                                         pcs, 
                                                         [b'g', b'r'],
                                                         mimic_alerts=mimic_alerts)
    train_features_df = append_y_true_col(features_df=train_features_df,
                                          prediction_type_nos=[149, 150, 151],
                                          meta_df = df_header, 
                                          meta_key_col_name = "SNID",
                                          meta_type_col_name = "SNTYPE")

100%|██████████| 22280/22280 [06:40<00:00, 55.62it/s]


In [8]:
train_features_df

,coeff1_g,coeff2_g,coeff3_g,residuo_g,maxflux_g,coeff1_r,coeff2_r,coeff3_r,residuo_r,maxflux_r,key,current_dates,type,type_names,y_true
0,0.632028,0.008088,0.366976,0.958710,230.670456,0.727304,-0.098875,0.355660,1.610522,373.748047,1757,57661.2736,150,150: KN GW170817,True
1,0.782480,0.268593,0.149629,0.466376,427.853790,1.060051,-0.059434,-0.106277,1.989556,788.683228,6415,57461.4561,141,141: 91BG,False
2,0.750644,0.087323,0.172727,0.937126,270.629425,0.876449,0.080352,0.104214,0.825639,401.693390,7707,57484.2005,103,103: Core collapse Type Ibc,False
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.712076,0.151259,0.263020,1.071781,534.019775,8267,57671.3147,151,151: KN Karsen 2017,True
4,1.028029,-0.020704,0.038843,0.993747,263.286224,0.000000,0.000000,0.000000,0.000000,0.000000,12578,57517.4403,102,102: MOSFIT-Ibc,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22275,1.088962,-0.014523,0.106654,1.693484,888.318420,1.205512,-0.059662,0.725981,1.578576,666.974243,137062467,57565.3666,112,112: Core collapse Type II,False
22276,0.978098,-0.118905,-0.022548,1.571079,1141.677979,1.038589,-0.084343,0.017370,1.344228,1213.934937,137071784,57690.5287,170,170: AGN,False
22277,1.038554,-0.092552,0.148199,1.327237,295.870941,0.990183,-0.038269,0.027255,0.717529,408.148193,137071978,57621.3001,143,143: Iax,False
22278,0.622255,-0.028716,0.439814,1.676503,525.301697,0.000000,0.000000,0.000000,0.000000,0.000000,137079473,57493.3158,151,151: KN Karsen 2017,True


# Now Train the classifier

In [9]:
from kndetect.training import train_classifier

In [10]:
if not use_already_trained_models:
    clf, features_df = train_classifier(train_features_df)
    if save_data:
        with open(os.path.join(data_dir, "models", sub_directory + ".pkl"), 'wb') as files:
            pickle.dump(clf, files)
else:
    from kndetect.predict import load_classifier
    clf = load_classifier(sub_directory + ".pkl")

/pbs/throng/lsst/users/bbiswas/miniconda3/envs/kndetect/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
/pbs/throng/lsst/users/bbiswas/miniconda3/envs/kndetect/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [11]:
# Generate Test features and Predict probabilities

In [12]:
df_header_test = Table.read(test_data_head_path, format='fits').to_pandas()
df_phot_test = Table.read(test_data_phot_path, format='fits').to_pandas()

In [13]:
if use_already_trained_features:
    test_features_df = pd.read_csv(os.path.join(data_dir, sub_directory, "test_features.csv"),index_col=0)

else:
    test_features_df = extract_features_all_lightcurves(df_phot_test, 
                                                         "SNID", 
                                                         pcs, 
                                                         [b'g', b'r'],
                                                         mimic_alerts=mimic_alerts)
    test_features_df = append_y_true_col(features_df=test_features_df,
                                          prediction_type_nos=[149, 150, 151],
                                          meta_df = df_header_test, 
                                          meta_key_col_name = "SNID",
                                          meta_type_col_name = "SNTYPE")

100%|██████████| 21288/21288 [06:32<00:00, 54.23it/s]


In [14]:
from kndetect.predict import load_classifier, predict_kn_score

In [15]:
probabilities, filtered_indices = predict_kn_score(clf, test_features_df)

In [16]:
test_features_df['y_pred_score'] = probabilities.T[1]

In [26]:
filtered=test_features_df[filtered_indices]

In [40]:
y_true = filtered["y_true"] == 1
y_pred = filtered["y_pred_score"]>0.5

In [43]:
print("True Positive : {tp}".format(tp = np.sum(y_true&y_pred)))
print("False Positive : {fp}".format(fp = np.sum(~y_true&y_pred)))
print("True negative : {tn}".format(tn = np.sum(~y_true&~y_pred)))
print("False negative : {fn}".format(fn = np.sum(y_true&~y_pred)))

True Positive : 750
False Positive : 370
True negative : 13484
False negative : 372


In [44]:
if save_data:
    train_features_df.to_csv(os.path.join(data_dir, sub_directory, "train_features.csv"))
    test_features_df.to_csv(os.path.join(data_dir, sub_directory, "test_features.csv"))